# Estimating Value Function in Gridworld Model

In the Gridworld model, there are 25 states. At each state, there are four available actions and reward is distributed according to the state and action pair.
In particular, there are two special states in the model. When one steps onto those two states, the state transition is regardless of the action taken and one gets higher immediate rewards.
All the details of the model is presented in the note and we do not mention further details on the setting of Gridworld in this notebook.

## Setting up the Game

In [1]:
import numpy as np
import pandas as pd
import math
from math import isclose
from collections import defaultdict

The state value is formed as a tuple $(i,j)$ where $i,j\in\left\{0,1,2,3,4\right\}$. The following function judges if a state tuple is legal.

In [2]:
# The function judges if the state is legal
def is_legal_state(state):
    if state[0] < -0.5 or state[0] > 4.5 or state[1] < -0.5 or state[1] > 4.5:
        return False
    return True

Given current state and action, the transition kernel decides what the next state is and how much immediate reward an agent receives. The transition kernel is modelled in the function below.

In [3]:
# Transition kernel of MDP
# Input: state as a tuple, action as a string
# Output: a real number as reward and a tuple as next state
def get_reward_state(state,action):
    # Two special states
    if state == (1,4):
        next_state = (1,0)
        reward = 10
        return reward, next_state
    elif state == (3,4):
        next_state = (3,2)
        reward = 5
        return reward, next_state

    # General cases
    if action == 'N':
        next_state = (state[0],state[1] + 1)
    elif action == 'S':
        next_state = (state[0],state[1] - 1)
    elif action == 'W':
        next_state = (state[0] - 1,state[1])
    elif action == 'E':
        next_state = (state[0] + 1,state[1])

    # Is the next state legal?
    if is_legal_state(next_state):
        reward = 0
    else:
        # Next state illegal, does not move and receive reward -1
        next_state = state
        reward = -1
    return reward, next_state

The following function simulates the MDP.

In [4]:
# Simulate Gridworld until given time limit
# Input: simulate until MAX_TIME, the initial state given
# Output: lists of state, action, reward as the experience
def run_gridworld(MAX_TIME,init_state):
    state_list = list()
    action_list = list()
    reward_list = list()

    # Initial state
    state = init_state

    for t in range(MAX_TIME):
        # Figure out the action, reward and next_state
        action = action_to_take(state)
        reward, next_state = get_reward_state(state,action)

        # Record the state,action,reward
        state_list.append(state)
        action_list.append(action)
        reward_list.append(reward)

        # Update the state
        state = next_state

    return state_list,action_list,reward_list

The return is calculated through $G_t = R_{t+1} + \gamma G_{t+1}$ in a backward style.

In [5]:
# The function computes the return
# Input: reward_list, gamma as discount rate
# Output: return_list, with length MAX_TIME - 1
def compute_return_list(MAX_TIME,reward_list,gamma = 0.9):
    return_list = [0] * (MAX_TIME - 1)

    # Calculate return backwardly
    return_list[-1] = reward_list[-1]
    for t in range(MAX_TIME - 3,-1,-1):
        return_list[t] = reward_list[t] + gamma * return_list[t + 1]

    return return_list

## Estimate Value Function for a Given Policy

The policy $\pi$ is given as the one that always chooses four actions with same probability $\frac{1}{4}$ regardless of the current state.
Let's estimate state value function $v_\pi$ through Monte Carlo, noticing that it has the structure as an expectation
$$v_\pi(s) = \mathbb{E}_\pi\left[G_t|S_t = s\right]$$

In [6]:
# Return action under policy \pi
def action_to_take(state):
    import numpy as np
    r = np.random.rand()
    if r < 1/4:
        return 'N'
    elif r < 1/2:
        return 'S'
    elif r < 3/4:
        return 'E'
    else:
        return 'W'

In [7]:
# Estimate v_\pi through Monte Carlo
# Input: NUM_CHAIN as the number of samples in MC
# Output: 5 * 5 matrix as the state value function
def est_v_pi(NUM_CHAIN,MAX_TIME):

    # State value function
    v_pi = np.zeros((5,5))

    # Store the returns for each state
    state_bins = defaultdict(list)

    for _ in range(NUM_CHAIN):
        # Randomly choose initial state
        init_x = np.random.randint(0,5)
        init_y = np.random.randint(0,5)
        init_state = (init_x,init_y)
        
        # Simulate
        state_list, action_list, reward_list = run_gridworld(MAX_TIME,init_state)

        # Compute return
        return_list = compute_return_list(MAX_TIME,reward_list)

        # If S_t = s, put G_t into the bin of state s
        for i in range(len(return_list) - 1,0,-1):
            state_bins[state_list[i]].append(return_list[i])

    # Compute mean
    for i in range(5):
        for j in range(5):
            v_pi[i,j] = np.mean(state_bins[(i,j)])

    return v_pi

Let's check the estimation result.

In [8]:
# Fix random seed
np.random.seed(1)

# Parameter
NUM_CHAIN = 1000
MAX_TIME = 20000

# Estimate and print
v_pi = est_v_pi(NUM_CHAIN,MAX_TIME)
print(pd.DataFrame(data = v_pi))

          0         1         2         3         4
0 -1.856261 -0.978066  0.042301  1.521796  3.311155
1 -1.344770 -0.435928  0.740045  2.994327  8.790195
2 -1.226271 -0.354464  0.675206  2.247773  4.421236
3 -1.421055 -0.587406  0.352294  1.902699  5.314808
4 -1.976971 -1.188604 -0.409326  0.544080  1.492610


## Estimate Optimal Value Function and Optimal Policy

The optimal value function $v^*$ is estimated through fixed point iteration thanks to the Bellman optimality operator being a contraction mapping.
$$v_{n+1}(s)=\sup_a \sum_{s',r}p(s',r|s,a)\cdot [r+\gamma\cdot v_n(s')]$$

From $v^*$, we can figure out the optimal policy $\pi^*$ by noticing that the deterministic optimal policy takes the action that achieves the $\sup$ in Bellman optimality equation for $v^*(s)$ when the current state is $s$.
We integrate the construction of optimal policy in the function below.

In [9]:
# Fixed point iteration to estimate the optimal state value function
# and get the optimal policy
# Input: error tolerance as stopping criterion is fixed point iteration
# Output: 5 * 5 matrix as v_star, a dictionary as optimal policy
def est_v_star(TOL,gamma = 0.9):
    # Iterate until the estimates are close enough
    last_est = np.ones((5,5))
    est = np.zeros((5,5))
    
    # Deterministic optimal policy
    opt_policy = dict()

    while np.linalg.norm(last_est - est,'fro') > TOL:
        last_est = est.copy()
        # Fixed point iteration for each state
        for i in range(5):
            for j in range(5):
                state = (i,j)
                # Update the value for state (i,j)
                
                # Deal with two special states (1,4), (3,4)
                if state == (1,4):
                    # Next state (1,0) with reward 10
                    est[i,j] = 10 + gamma * last_est[1,0]
                    opt_policy[state] = 'NSWE'
                elif state == (3,4):
                    # Next state (3,2) with reward 5
                    est[i,j] = 5 + gamma * last_est[3,2]
                    opt_policy[state] = 'NSWE'
                else:
                    # Regular cases
                    # four possibilities for next_state 
                    next_state_list = [(state[0],state[1] + 1),(state[0],state[1] - 1)
                                      ,(state[0] - 1,state[1]),(state[0] + 1,state[1])]
                    value_list = list()
                    for next_state in next_state_list:
                        if is_legal_state(next_state):
                            # Legal next state, reward 0
                            value = gamma * last_est[next_state[0],next_state[1]]
                        else:
                            # Illegal next state, reward -1
                            value = -1 + gamma * last_est[i,j]
                        # Append those values to a list
                        value_list.append(value)
                    # Take the maximum among those four values
                    est[i,j] = np.max(value_list)
                    
                    # Record which action achieves the max
                    action_list = ['N','S','W','E']
                    best_action = ''
                    for k in range(len(action_list)):
                            if isclose(est[i,j],value_list[k],abs_tol = 1e-4):
                                # Record all best actions for this state
                                best_action = best_action + action_list[k]
                    # Update the optimal policy
                    opt_policy[state] = best_action
                    
    return est, opt_policy

Let's derive the estimate for $v^*$.

In [10]:
TOL = 1e-4
v_star, opt_policy = est_v_star(TOL)
print(pd.DataFrame(data = v_star))

           0          1          2          3          4
0  14.419349  16.021499  17.801666  19.779673  21.977414
1  16.021499  17.801666  19.779673  21.977414  24.419349
2  14.419349  16.021499  17.801666  19.779673  21.977414
3  12.977414  14.419349  16.021499  17.801666  19.419349
4  11.679673  12.977414  14.419349  16.021499  17.477414


In [11]:
# Print optimal policy
print(pd.DataFrame(data = opt_policy.items()))

         0     1
0   (0, 0)    NE
1   (0, 1)    NE
2   (0, 2)    NE
3   (0, 3)    NE
4   (0, 4)     E
5   (1, 0)     N
6   (1, 1)     N
7   (1, 2)     N
8   (1, 3)     N
9   (1, 4)  NSWE
10  (2, 0)    NW
11  (2, 1)    NW
12  (2, 2)    NW
13  (2, 3)    NW
14  (2, 4)     W
15  (3, 0)    NW
16  (3, 1)    NW
17  (3, 2)    NW
18  (3, 3)     W
19  (3, 4)  NSWE
20  (4, 0)    NW
21  (4, 1)    NW
22  (4, 2)    NW
23  (4, 3)     W
24  (4, 4)     W


In this notebook, we shown how to compute the value function for a specific policy from the experience of the game (model-free), and how to estimate the optimal value function, optimal policy on knowing the transition kernel (model-based).
Those are the fundamentals of the RL algorithm we will build in a later context.